In [18]:
import arcpy
from arcpy.sa import *
import os
import sys
import numpy as np
from pathlib import Path

import configparser

config = configparser.ConfigParser()
config.read(r"H:\\variable.ini")



['H:\\\\variable.ini']

In [19]:
arcpy.env.workspace = config['PATH']['workspace_path']

arcpy.env.overwriteOutput = True


# Define the directory containing the TIFF files
tiff_directory = config['PATH']['tiff_directory']

path_to_raster=config['PATH']['input_path']
output=config['VARIABLE']['output_name']


In [22]:
#print("Feature Classes in the Geodatabase:")
#for feature_class in feature_classes:
#   print(feature_class)

def tiff_to_polygon(tiff_path,output_polygon_path,simplify=False):
    """
    Converts a TIFF file to a polygon vector using ArcPy.

    Parameters:
    - tiff_path: The file path of the TIFF file.
    - output_polygon_path: The path where the output polygon will be saved.
    - simplify: Boolean indicating whether to simplify the polygons.
    """
    try:
        # Check out the ArcGIS Spatial Analyst extension license
        arcpy.CheckOutExtension("Spatial")

        
        print(f"checked extentions")
        # load raster
        raster = arcpy.Raster(tiff_path)

        #mutiple each cell by 1000000
        multiplied_raster=raster*1000000
        #multiplied_raster.save("name the feature that you want to save")
        #Convert to integer
        int_raster=Int(multiplied_raster)

        # Convert the raster to a polygon feature class
        arcpy.RasterToPolygon_conversion(int_raster, output_polygon_path, simplify, "VALUE")
        
        print(f"Polygon vector created successfully at {output_polygon_path}")
        
        #divide by 1000000 
    
        # Set the field on which to perform the calculation
        field_name = "gridcode"

        # Define the expression to divide each value by 1,000,000
        # The '!field_name!' syntax is used to reference the field values in the calculation
        expression = f"!{field_name}! / 1000000"

        # Perform the field calculation
        arcpy.CalculateField_management(in_table=output, field=field_name, expression=expression, expression_type="PYTHON3")
        print("Field calculation completed successfully.")
        
        # Specify the name of the field 
        old_field_name = "gridcode"

        # Specify the new name for the field
        new_field_name = "value"

        # Specify the new alias for the field
        new_field_alias = "value"

        # Use the AlterField_management tool to rename the field and change its alias
        arcpy.AlterField_management(in_table=output, field=old_field_name, 
                                    new_field_name=new_field_name, new_field_alias=new_field_alias)

        print(f"Field '{old_field_name}' has been renamed to '{new_field_name}' with the alias '{new_field_alias}'.")


        

    except Exception as e:
        print(f"Error converting TIFF to polygon: {e}")
    finally:
        # Check in the ArcGIS Spatial Analyst extension license
        arcpy.CheckInExtension("Spatial")



In [7]:
tiff_to_polygon(path_to_raster,output,simplify=False)



checked extentions
Polygon vector created successfully at Skagit_heat_map_updated
Field calculation completed successfully.
Field 'gridcode' has been renamed to 'value' with the alias 'value'.


In [23]:
#for filename in os.listdir(tiff_directory):
   # if filename.endswith(".tif"):
        # Construct the full file path
       # tiff_path = os.path.join(tiff_directory, filename)

        #print(tiff_path)

        # Generate a unique output path for the polygon
       # output_name = os.path.splitext(filename)[0] + "_polygon"
        
      #  output_polygon_path = os.path.join(arcpy.env.workspace, output_name)
       # print(output_polygon_path)

        # Call the conversion function
      #  tiff_to_polygon(tiff_path, output_polygon_path,simplify=False)

      #  print(f"Processed {filename}")


#not working yet